In [1]:
import rdflib
from rdflib import Graph, Literal, RDF, RDFS, URIRef, Namespace
from rdflib.namespace import FOAF, XSD
import pandas as pd
import numpy as np
import re 
import requests
from bs4 import BeautifulSoup

In [2]:
from rdflib import Graph

In [34]:
#Defining the namespace
nspace = Namespace("Project.rdf")

# Create a Graph
g = Graph()

# Parse in an RDF file hosted locally
g.parse("Project2.rdf")

g.namespace_manager.bind("nl",nspace)

# Loop through each triple in the graph (subj, pred, obj)
for subj, pred, obj in g:
    # Check if there is at least one triple in the Graph
    if (subj, pred, obj) not in g:
       raise Exception("It better be!")

# Print the number of "triples" in the Graph
print(f"Graph g has {len(g)} statements.")

# Print out the entire Graph in the RDF Turtle format
print(g.serialize(format="n3"))

Graph g has 231 statements.
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix webSem: <http://www.semanticweb.org/marya/ontologies/2023/2/webSem#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://www.semanticweb.org/marya/ontologies/2023/2/webSem> a owl:Ontology .

webSem:Actor a owl:Class ;
    rdfs:subClassOf webSem:Person ;
    owl:equivalentClass [ a owl:Class ;
            owl:intersectionOf ( webSem:Person [ a owl:Restriction ;
                        owl:onProperty webSem:isActorOf ;
                        owl:someValuesFrom webSem:Movie ] ) ] .

webSem:BTime2 a webSem:BroadcastTime,
        owl:NamedIndividual ;
    webSem:endDate "2023-03-20T00:00:00"^^xsd:dateTime ;
    webSem:startDate "2023-03-08T00:00:00"^^xsd:dateTime .

webSem:DanielCraig a webSem:Person,
        owl:NamedIndividual ;
    webSem:age "55"^^xsd:int ;
    webSem:isActorOf 

In [4]:
qres = g.query("""
                SELECT ?title
                    WHERE { ?x rdf:type webSem:Movie;
                               webSem:title ?title }
                """)

for row in qres:
    print(f"{row.title}")

Casino Royale
Kill Bill
Pulp Fiction


In [5]:
qres = g.query("""
                SELECT ?title ?year
                    WHERE { ?x rdf:type webSem:Movie.
                            ?x webSem:title ?title.
                            ?x webSem:year ?year
                               }
                """)

for row in qres:
    print(f"{row.title}, {row.year}")

Casino Royale, 2006
Kill Bill, 2003
Pulp Fiction, 1994


In [33]:
qres = g.query("""
                SELECT DISTINCT ?name ?age ?title ?genre
                    WHERE { ?x webSem:isActorOf ?movie.
                            ?movie webSem:title ?title.
                            ?movie webSem:genre ?genre.
                            ?x webSem:name ?name.
                            ?x webSem:age ?age
                            FILTER regex(?genre, "Crime")
                               }
                """)

for row in qres:
    print(f"{row.name}, {row.age}, {row.title}, {row.genre}")


John Travolta, 59, Pulp Fiction, Crime Thriller
Uma Thurman, 43, Pulp Fiction, Crime Thriller
Quentin Tarantino, 53, Kill Bill, Action Crime Thriller


In [23]:
qres = g.query("""
                SELECT DISTINCT ?name ?age
                    WHERE { ?x webSem:isDirectorOf ?isDirectorOf.
                            ?x webSem:name ?name.
                            ?x webSem:age ?age
                               }
                """)

for row in qres:
    print(f"{row.name}, {row.age}")

Quentin Tarantino, 53


In [ ]:
qres = g.query("""
                SELECT DISTINCT ?name ?age
                    WHERE { ?x webSem:isDirectorOf ?isDirectorOf.
                            ?x webSem:name ?name.
                            ?x webSem:age ?age
                               }
                """)

for row in qres:
    print(f"{row.name}, {row.age}")

In [46]:
qres = g.query("""
                SELECT  ?title ?genre
                    WHERE { ?x webSem:title ?title.
                            ?x webSem:genre ?genre
                            FILTER (?title="Kill Bill")
                               }
                """)

print(qres)
title = ""
genre = ""
for row in qres:
    title = row.title
    genre  row.genre + " "

print(f"{row.title} {row.genre}")

Kill Bill Crime
